## Bayesian optimization demo

This notebook demonstrates using Bayesian optimization to find the maximum value of a function $f$, with limited trials. 

The goal is to find the x-point corresponding to the highest y-value, given 10 trials. 

Bayesian optimization is a way to balance exploration-exploitation trade-off, while we search the parameter space (x-axis). 

The idea can be broken down into two components:

1) Applying Gaussian Process to measure the similarity between the x-points, and constructe the posterior mean and variance of the y-values.

2) Given the mean and variance at each candidate point, we can device an "acquisition function" to guide us which point should we pick 


The two widely used acquisition functions are i) Expected Improvement (EI) and ii) Upper Confidence Bound (UCB)

One thing to point out is that, we are looking for maximum of the function values, but scipy's optimize routine is `minimize`, so we will define our acquisition function as negative expected improvement and negative upper condifence bound, respectively. So that we can throw the defined acquisition function into `scipy.optimize.minimize` routine.

Also, to apply Gaussian Process, we will need an kernel function to measure the similarity between two x-points. We will use Radial Basis Function (RBF) kernel.

In [1]:
import numpy as np
from bayesian_optimization import BayesianOptimization, BayesianOptimization1dDemo

## Define a function

In [2]:
def f(x):
    return -(3.671 * x - 1.956)**2 * np.sin(9.5 * x - 3.6)

### Part I, acquisition function = (negative) Expected Improvement

At a given iteration, the imporvement can be written as 

$
I = max(y - y^*, 0),
$

where $y^*$ is the max value seen so far. $I$ is a Gaussian variable with mean $\mu$ and standard deviation $\sigma$. 
The expected improvement is simply the expected value of $I$, $E [I]$, which can be derived in closed form and written in terms of the cdf and pdf of a standard normal.

In [5]:
np.random.seed(98765)
bo = BayesianOptimization1dDemo(f, np.array([[0.0, 1.0]]), policy='ei')
bo.fit()
optimal = bo.optimal()
print("\noptimal at: {0:}, value: {1:}".format(optimal[0], optimal[1]))

fmin_l_bfgs_b terminated abnormally. State: {'grad': array([ -1.58189676e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 8, 'warnflag': 2}

optimal at: [ 0.9155699], value: 1.8292824444999078


In [ ]:
# convert -delay 100 -loop 0 -dispose previous ei_iteration_0*.png ei.gif

The plot below shows the progress of the search process. The red triangle markes the point selected as the next move, as guided by the acquisition function. In most cases the selected point corresponds to the lowest point of the acquisition function at that iteration. There are some exceptions, for example, iteration 8, which selected the local minima as the next move. This is most likely because in the implementation, we use `L-BFGS-B` algorithm to find the minimum, and the algorithm might be trapped in the local minima.

An alternative to avoid this issue would be to compute the function values on a pre-selcted grid on x-axis. The computed value would be based on Gaussian Process posterior. 

In [6]:
from IPython.display import HTML
HTML('<img src="ei.gif">')

### Part II, acquisition function = (negative) Upper Confidence Bound (UCB),  where UCB = $\mu + \lambda \sigma$

$\mu$ is the mean at a given position, and $\sigma$ the standard deviation at the same poistion. 
If we want to do exploitation, we should go with where $\mu$ is high. On the contrary, if we want to do exploration, we should go with where $\sigma$ is high. The parameter $\lambda$ hence decide the trad-off between exploration and exploitation. In the code $\lambda$ is set to be 1.5.

In [8]:
np.random.seed(98765)
bo = BayesianOptimization1dDemo(f, np.array([[0.0, 1.0]]), policy='ucb')
bo.fit()
optimal = bo.optimal()
print("\noptimal at: {0:}, value: {1:}".format(optimal[0], optimal[1]))


optimal at: [ 0.91513982], value: 1.8281848592343553


In [ ]:
# convert -delay 100 -loop 0 -dispose previous ucb_iteration_0*.png ucb.gif

Despite very different acquisition function, `UCB` gives us the same optimal result as `EI` did.

In [9]:
from IPython.display import HTML
HTML('<img src="ucb.gif">')